In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import seaborn as sns
import statsmodels.api as sm

font_path = 'C:\\Windows\\Fonts\\gulim.ttc'
font = fm.FontProperties(fname=font_path).get_name()
mpl.rc('font', family=font)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

def divide_time_range(time):
    if time < 600:
        return '00:00~06:00'
    elif time < 1000:
        return '06:00~10:00' # 출근시간
    elif time < 1700:
        return '10:00~17:00'
    elif time < 2100:
        return '17:00~21:00' # 퇴근시간
    else:
        return '21:00~24:00'

def check_best_arima(data):
    best_aic = float("inf")  # 초기값 설정
    best_bic = float("inf")  # 초기값 설정
    best_model = None
    best_p, best_d, best_q = None, None, None
    
    for p in range(1, 6):
        for d in range(1, 3):
            for q in range(0, 6):
                try:
                    model = ARIMA(data, order=(p, d, q))
                    model_fit = model.fit()
                    aic = model_fit.aic
                    bic = model_fit.bic
                    if bic < best_bic or (bic == best_bic and aic < best_aic):
                        best_aic = aic
                        best_bic = bic
                        best_model = model_fit
                        best_p, best_d, best_q = p, d, q
                except:
                    continue
    
    return best_model, best_p, best_d, best_q

def arima_test(data, IDs):
    for 주거_ID in IDs:
        주거_data = data[data['대여소_ID'] == 주거_ID]
        주거_data['기준_날짜'] = pd.to_datetime(주거_data['기준_날짜'])
        주거_data.set_index('기준_날짜', inplace=True)
        arima_data = 주거_data.resample('D')['전체_건수'].sum()
        
        best_model, best_p, best_d, best_q = check_best_arima(arima_data)
        if best_model is not None:
            print(f"대여소 {주거_ID}의 best model (p, d, q): ({best_p}, {best_d}, {best_q})")
            forecast = best_model.forecast(steps=1)  # 다음 시간대에 대한 예측값
            print(f"대여소 {주거_ID}의 다음 시간대 예측값: {forecast}")



In [4]:
강서_대여소_ID = pd.read_csv('../../Data/따릉이/강서_따릉이_대여소_ID.csv',encoding='utf-8')

강서_데이터1 = pd.read_csv('../../Data/따릉이/강서_따릉이_데이터_01.csv',encoding='utf-8')
강서_데이터2 = pd.read_csv('../../Data/따릉이/강서_따릉이_데이터_02.csv',encoding='utf-8')

강서_데이터 = pd.concat([강서_데이터1, 강서_데이터2], ignore_index=True)
print(len(강서_데이터))

top_20_ID = pd.read_csv('../../Data/따릉이/쏠림정도_top20_대여소.csv',encoding='utf-8')
# 주거와 상업으로 데이터 분할
주거_df = top_20_ID[top_20_ID['최종용도'] == '주거']
상업_df = top_20_ID[top_20_ID['최종용도'] == '상업']

# 주거 데이터에서 대여소_ID 추출
주거_IDs = 주거_df['대여소_ID'].unique()

# 상업 데이터에서 대여소_ID 추출
상업_IDs = 상업_df['대여소_ID'].unique()

강서_df = 강서_데이터[강서_데이터['종료_대여소_ID'].isin(주거_IDs)] # 시작 : 대여 ,종료 : 반납   + 주거, 상업
강서_df = 강서_df.sort_values(by='기준_날짜')

강서_df['시간대_구간'] = 강서_df['기준_시간대'].apply(divide_time_range)
강서_df1 = 강서_df[강서_df['시간대_구간'] == '06:00~10:00']  # 06~10 : , 17:21 :퇴근
강서_df1 = 강서_df1[(강서_df1['전체_이용_분'] != 0) & (강서_df1['전체_이용_거리'] != 0)]

강서_df1 = 강서_df1[['종료_대여소_ID', '기준_날짜', '전체_건수']].copy()  # 시작 : 대여 ,종료 : 반납
강서_df1 = 강서_df1.rename(columns={'종료_대여소_ID': '대여소_ID'}) # 시작 : 대여 ,종료 : 반납

arima_test(강서_df1,주거_IDs) # 주거 or 상업

3385613


C:\Users\DW\AppData\Local\Temp\ipykernel_7524\3542165546.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  주거_data['기준_날짜'] = pd.to_datetime(주거_data['기준_날짜'])
c:\Users\DW\.conda\envs\dw_test1\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\DW\.conda\envs\dw_test1\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\DW\.conda\envs\dw_test1\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non

대여소 ST-1253의 best model (p, d, q): (5, 1, 5)
대여소 ST-1253의 다음 시간대 예측값: 2024-03-30    0.415404
Freq: D, dtype: float64


c:\Users\DW\.conda\envs\dw_test1\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\DW\.conda\envs\dw_test1\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\DW\.conda\envs\dw_test1\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\DW\.conda\envs\dw_test1\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\DW\.conda\envs\dw_test1

대여소 ST-1352의 best model (p, d, q): (4, 1, 5)
대여소 ST-1352의 다음 시간대 예측값: 2024-03-30    1.119464
Freq: D, dtype: float64


...

c:\Users\DW\.conda\envs\dw_test1\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\DW\.conda\envs\dw_test1\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\DW\.conda\envs\dw_test1\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\DW\.conda\envs\dw_test1\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\DW\.co

대여소 ST-2557의 best model (p, d, q): (4, 1, 5)
대여소 ST-2557의 다음 시간대 예측값: 2024-03-30    4.632153
Freq: D, dtype: float64


c:\Users\DW\.conda\envs\dw_test1\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\DW\.conda\envs\dw_test1\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\DW\.conda\envs\dw_test1\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\DW\.conda\envs\dw_test1\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\DW\.co

대여소 ST-2965의 best model (p, d, q): (4, 1, 5)
대여소 ST-2965의 다음 시간대 예측값: 2024-03-30    2.085584
Freq: D, dtype: float64


c:\Users\DW\.conda\envs\dw_test1\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [6]:
강서_대여소_ID = pd.read_csv('../../Data/따릉이/강서_따릉이_대여소_ID.csv',encoding='utf-8')

강서_데이터1 = pd.read_csv('../../Data/따릉이/강서_따릉이_데이터_01.csv',encoding='utf-8')
강서_데이터2 = pd.read_csv('../../Data/따릉이/강서_따릉이_데이터_02.csv',encoding='utf-8')

강서_데이터 = pd.concat([강서_데이터1, 강서_데이터2], ignore_index=True)
print(len(강서_데이터))

top_20_ID = pd.read_csv('../../Data/따릉이/쏠림정도_top20_대여소.csv',encoding='utf-8')
# 주거와 상업으로 데이터 분할
주거_df = top_20_ID[top_20_ID['최종용도'] == '주거']
상업_df = top_20_ID[top_20_ID['최종용도'] == '상업']

# 주거 데이터에서 대여소_ID 추출
주거_IDs = 주거_df['대여소_ID'].unique()

# 상업 데이터에서 대여소_ID 추출
상업_IDs = 상업_df['대여소_ID'].unique()

강서_df = 강서_데이터[강서_데이터['종료_대여소_ID'].isin(상업_IDs)] # 시작 : 대여 ,종료 : 반납   + 주거, 상업
강서_df = 강서_df.sort_values(by='기준_날짜')

강서_df['시간대_구간'] = 강서_df['기준_시간대'].apply(divide_time_range)
강서_df1 = 강서_df[강서_df['시간대_구간'] == '06:00~10:00']  # 06~10 : , 17:21 :퇴근
강서_df1 = 강서_df1[(강서_df1['전체_이용_분'] != 0) & (강서_df1['전체_이용_거리'] != 0)]

강서_df1 = 강서_df1[['종료_대여소_ID', '기준_날짜', '전체_건수']].copy()  # 시작 : 대여 ,종료 : 반납
강서_df1 = 강서_df1.rename(columns={'종료_대여소_ID': '대여소_ID'}) # 시작 : 대여 ,종료 : 반납

arima_test(강서_df1,상업_IDs) # 주거 or 상업

3385613


C:\Users\DW\AppData\Local\Temp\ipykernel_7524\3542165546.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  주거_data['기준_날짜'] = pd.to_datetime(주거_data['기준_날짜'])
c:\Users\DW\.conda\envs\dw_test1\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\DW\.conda\envs\dw_test1\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\DW\.conda\envs\dw_test1\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978:

대여소 ST-1064의 best model (p, d, q): (5, 1, 5)
대여소 ST-1064의 다음 시간대 예측값: 2024-03-30    15.731446
Freq: D, dtype: float64


c:\Users\DW\.conda\envs\dw_test1\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\DW\.conda\envs\dw_test1\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\DW\.conda\envs\dw_test1\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\DW\.conda\envs\dw_test1\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\DW\.co

대여소 ST-1688의 best model (p, d, q): (5, 1, 5)
대여소 ST-1688의 다음 시간대 예측값: 2024-03-30    11.658823
Freq: D, dtype: float64


...

c:\Users\DW\.conda\envs\dw_test1\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\DW\.conda\envs\dw_test1\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\DW\.conda\envs\dw_test1\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\DW\.conda\envs\dw_test1\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\DW\.conda\envs\

대여소 ST-3128의 best model (p, d, q): (4, 1, 5)
대여소 ST-3128의 다음 시간대 예측값: 2024-03-30    8.49684
Freq: D, dtype: float64


c:\Users\DW\.conda\envs\dw_test1\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\DW\.conda\envs\dw_test1\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\DW\.conda\envs\dw_test1\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\DW\.conda\envs\dw_test1\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\DW\.co

대여소 ST-3251의 best model (p, d, q): (5, 1, 5)
대여소 ST-3251의 다음 시간대 예측값: 2024-03-30    11.169073
Freq: D, dtype: float64


c:\Users\DW\.conda\envs\dw_test1\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [40]:
# 주거 데이터에서 대여소_ID 추출
주거_IDs = 주거_df['대여소_ID'].unique()

# 상업 데이터에서 대여소_ID 추출
상업_IDs = 상업_df['대여소_ID'].unique()

# 강서 데이터에서 대여소_ID가 주거 데이터에 있는 것만 추출
주거_강서_df = 강서_데이터[강서_데이터['시작_대여소_ID'].isin(주거_IDs)]
주거_강서_df1 = 강서_데이터[강서_데이터['종료_대여소_ID'].isin(주거_IDs)]

# 강서 데이터에서 대여소_ID가 상업 데이터에 있는 것만 추출
상업_강서_df = 강서_데이터[강서_데이터['시작_대여소_ID'].isin(상업_IDs)]
상업_강서_df1 = 강서_데이터[강서_데이터['종료_대여소_ID'].isin(상업_IDs)]

array(['ST-1253', 'ST-1352', 'ST-1645', 'ST-2050', 'ST-2056', 'ST-2557',
       'ST-2965'], dtype=object)

In [52]:
# 주거와 상업으로 데이터 분할
주거_df = top_20_ID[top_20_ID['최종용도'] == '주거']
상업_df = top_20_ID[top_20_ID['최종용도'] == '상업']

# 주거 데이터에서 대여소_ID 추출
주거_IDs = 주거_df['대여소_ID'].unique()

# 상업 데이터에서 대여소_ID 추출
상업_IDs = 상업_df['대여소_ID'].unique()

주거_강서_df = 강서_데이터[강서_데이터['시작_대여소_ID'].isin(주거_IDs)]
주거_강서_df = 주거_강서_df.sort_values(by='기준_날짜')

주거_강서_df['시간대_구간'] = 주거_강서_df['기준_시간대'].apply(divide_time_range)
주거_강서_df_출근 = 주거_강서_df[주거_강서_df['시간대_구간'] == '06:00~10:00']
주거_강서_df_출근 = 주거_강서_df_출근[(주거_강서_df_출근['전체_이용_분'] != 0) & (주거_강서_df_출근['전체_이용_거리'] != 0)]

arima_test(주거_강서_df1,주거_IDs)


C:\Users\ciw96\AppData\Local\Temp\ipykernel_19812\2229165998.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  주거_data['기준_날짜'] = pd.to_datetime(주거_data['기준_날짜'])
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\ciw96\AppData\Local\Programs\Python\

대여소 ST-1253의 best model (p, d, q): (5, 1, 2)
대여소 ST-1253의 다음 시간대 예측값: 2024-03-29    0.548459
Freq: D, dtype: float64


c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. 

대여소 ST-1352의 best model (p, d, q): (5, 1, 2)
대여소 ST-1352의 다음 시간대 예측값: 2024-03-30   -1.736504
Freq: D, dtype: float64


c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. 

...

c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using

대여소 ST-2557의 best model (p, d, q): (5, 1, 2)
대여소 ST-2557의 다음 시간대 예측값: 2024-03-30    0.74331
Freq: D, dtype: float64


c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. 

대여소 ST-2965의 best model (p, d, q): (5, 1, 1)
대여소 ST-2965의 다음 시간대 예측값: 2024-03-30    3.344337
Freq: D, dtype: float64


c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [53]:
# 주거와 상업으로 데이터 분할
주거_df = top_20_ID[top_20_ID['최종용도'] == '주거']
상업_df = top_20_ID[top_20_ID['최종용도'] == '상업']

# 주거 데이터에서 대여소_ID 추출
주거_IDs = 주거_df['대여소_ID'].unique()

# 상업 데이터에서 대여소_ID 추출
상업_IDs = 상업_df['대여소_ID'].unique()

주거_강서_df = 강서_데이터[강서_데이터['시작_대여소_ID'].isin(상업_IDs)]
주거_강서_df = 주거_강서_df.sort_values(by='기준_날짜')

주거_강서_df['시간대_구간'] = 주거_강서_df['기준_시간대'].apply(divide_time_range)
주거_강서_df_출근 = 주거_강서_df[주거_강서_df['시간대_구간'] == '06:00~10:00']
주거_강서_df_출근 = 주거_강서_df_출근[(주거_강서_df_출근['전체_이용_분'] != 0) & (주거_강서_df_출근['전체_이용_거리'] != 0)]

arima_test(주거_강서_df1,상업_IDs)


C:\Users\ciw96\AppData\Local\Temp\ipykernel_19812\2229165998.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  주거_data['기준_날짜'] = pd.to_datetime(주거_data['기준_날짜'])
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\ciw96\AppData\Local\Programs\Python\

대여소 ST-1064의 best model (p, d, q): (5, 1, 2)
대여소 ST-1064의 다음 시간대 예측값: 2024-03-30    7.889067
Freq: D, dtype: float64


c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using

대여소 ST-1688의 best model (p, d, q): (5, 1, 2)
대여소 ST-1688의 다음 시간대 예측값: 2024-03-30    0.679388
Freq: D, dtype: float64


c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. 

...

c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting paramet

대여소 ST-3128의 best model (p, d, q): (2, 2, 2)
대여소 ST-3128의 다음 시간대 예측값: 2024-03-30   -5624.335521
Freq: D, dtype: float64


c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using

대여소 ST-3251의 best model (p, d, q): (5, 1, 2)
대여소 ST-3251의 다음 시간대 예측값: 2024-03-30    0.904245
Freq: D, dtype: float64


c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [32]:
주거_강서_df = 주거_강서_df.sort_values(by='기준_날짜')
주거_강서_df1 = 주거_강서_df.sort_values(by='기준_날짜')
상업_강서_df = 상업_강서_df.sort_values(by='기준_날짜')
상업_강서_df1 = 상업_강서_df.sort_values(by='기준_날짜')

In [33]:
    
주거_강서_df['시간대_구간'] = 주거_강서_df['기준_시간대'].apply(divide_time_range)
주거_강서_df1['시간대_구간'] = 주거_강서_df1['기준_시간대'].apply(divide_time_range)
상업_강서_df['시간대_구간'] = 상업_강서_df['기준_시간대'].apply(divide_time_range)
상업_강서_df1['시간대_구간'] = 상업_강서_df1['기준_시간대'].apply(divide_time_range)



주거_강서_df_출근 = 주거_강서_df[주거_강서_df['시간대_구간'] == '06:00~10:00']
주거_강서_df_퇴근 = 주거_강서_df1[주거_강서_df['시간대_구간'] == '06:00~10:00']
상업_강서_df_출근 = 주거_강서_df[주거_강서_df['시간대_구간'] == '06:00~10:00']
상업_강서_df_퇴근 = 주거_강서_df1[주거_강서_df['시간대_구간'] == '06:00~10:00']
# 전체 파일에서 1차 전처리 (집계_기준 2개 -> 1개로 줄임, 사용량 =0)
주거_강서_df_출근 = 주거_강서_df_출근[(주거_강서_df_출근['전체_이용_분'] != 0) & (주거_강서_df_출근['전체_이용_거리'] != 0)]
주거_강서_df_퇴근 = 주거_강서_df_퇴근[(주거_강서_df_퇴근['전체_이용_분'] != 0) & (주거_강서_df_퇴근['전체_이용_거리'] != 0)]
상업_강서_df_출근 = 상업_강서_df_출근[(상업_강서_df_출근['전체_이용_분'] != 0) & (상업_강서_df_출근['전체_이용_거리'] != 0)]
상업_강서_df_퇴근 = 상업_강서_df_퇴근[(상업_강서_df_퇴근['전체_이용_분'] != 0) & (상업_강서_df_퇴근['전체_이용_거리'] != 0)]


C:\Users\ciw96\AppData\Local\Temp\ipykernel_19812\1946588316.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  주거_강서_df_퇴근 = 주거_강서_df1[주거_강서_df['시간대_구간'] == '06:00~10:00']
C:\Users\ciw96\AppData\Local\Temp\ipykernel_19812\1946588316.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  상업_강서_df_퇴근 = 주거_강서_df1[주거_강서_df['시간대_구간'] == '06:00~10:00']


In [18]:
grouped_data = 주거_강서_df1.groupby(['기준_날짜', '시작_대여소_ID'])['전체_건수'].sum().reset_index()
print(grouped_data)

           기준_날짜 시작_대여소_ID  전체_건수
0     2023-04-03   ST-1253     27
1     2023-04-03   ST-1352     56
2     2023-04-03   ST-1645     70
3     2023-04-03   ST-2056     61
4     2023-04-03   ST-2557     49
...          ...       ...    ...
1637  2024-03-29   ST-1645     62
1638  2024-03-29   ST-2050     75
1639  2024-03-29   ST-2056     46
1640  2024-03-29   ST-2557     57
1641  2024-03-29   ST-2965     58

[1642 rows x 3 columns]


# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [21]:
arima_test(주거_강서_df1,주거_IDs)

C:\Users\ciw96\AppData\Local\Temp\ipykernel_19812\385166018.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  주거_data['기준_날짜'] = pd.to_datetime(주거_data['기준_날짜'])
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\ciw96\AppData\Local\Programs\Python\P

                               SARIMAX Results                                
Dep. Variable:                  전체_건수   No. Observations:                  362
Model:                 ARIMA(1, 1, 1)   Log Likelihood               -1693.346
Date:                Wed, 08 May 2024   AIC                           3392.693
Time:                        09:56:09   BIC                           3404.359
Sample:                    04-03-2023   HQIC                          3397.331
                         - 03-29-2024                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.3838      0.061      6.304      0.000       0.264       0.503
ma.L1         -0.9896      0.012    -81.155      0.000      -1.014      -0.966
sigma2       688.8429     88.216      7.809      0.0

c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


                               SARIMAX Results                                
Dep. Variable:                  전체_건수   No. Observations:                  362
Model:                 ARIMA(2, 1, 1)   Log Likelihood               -1674.920
Date:                Wed, 08 May 2024   AIC                           3357.839
Time:                        09:56:09   BIC                           3373.395
Sample:                    04-03-2023   HQIC                          3364.024
                         - 03-29-2024                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.4874      0.072      6.809      0.000       0.347       0.628
ar.L2         -0.3179      0.066     -4.846      0.000      -0.447      -0.189
ma.L1         -0.9733      0.016    -59.535      0.0

c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


...

c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                               SARIMAX Results                                
Dep. Variable:                  전체_건수   No. Observations:                  332
Model:                 ARIMA(5, 1, 2)   Log Likelihood               -1504.697
Date:                Wed, 08 May 2024   AIC                           3025.395
Time:                        09:56:39   BIC                           3055.812
Sample:                    05-03-2023   HQIC                          3037.526
                         - 03-29-2024                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.3673      0.064      5.723      0.000       0.242       0.493
ar.L2         -0.6958      0.057    -12.204      0.000      -0.808      -0.584
ar.L3         -0.2617      0.062     -4.240      0.0

c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


                               SARIMAX Results                                
Dep. Variable:                  전체_건수   No. Observations:                  332
Model:                 ARIMA(5, 2, 2)   Log Likelihood               -1519.623
Date:                Wed, 08 May 2024   AIC                           3055.246
Time:                        09:56:40   BIC                           3085.639
Sample:                    05-03-2023   HQIC                          3067.369
                         - 03-29-2024                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.0902      0.065     -1.384      0.166      -0.218       0.037
ar.L2         -0.4697      0.045    -10.498      0.000      -0.557      -0.382
ar.L3         -0.3884      0.054     -7.241      0.0

c:\Users\ciw96\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------